In [1]:
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.datasets import mnist
import itertools
import random

Using TensorFlow backend.


In [2]:
# Randomly select a defined number of image paires from a given dataset
def get_left_min_image_combinations(number_image_pairs_selected, image_data, image_labels):
    
    num_images_total = len(image_labels)
    
    image_combination_data = []
    image_combination_labels = []
    
    for i in range(number_image_pairs_selected):
        
        # Draw two image indices from a uniform random distribution
        random_index_A = random.randint(0, num_images_total-1)
        random_index_B = random.randint(0, num_images_total-1)
        
        # Randomly choose two images from the dataset
        image_A = image_data[random_index_A]
        image_B = image_data[random_index_B]
        
        # Merge images TODO: this could be a function with several options
        combined_images = np.hstack((image_A, image_B))
        
        # Find the minimum between the two labels
        label_A = image_labels[random_index_A]
        label_B = image_labels[random_index_B]
        minimum_label = min(label_A, label_B)
        
        if label_A < label_B:
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_A, image_B))
        elif label_A > label_B: 
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_B, image_A))
        # Here we want to exclude the images where they are the same on both sides
            
   
        # Append newly generated image combination and minimum label to list
        image_combination_data.append(combined_images)
        image_combination_labels.append(minimum_label)
        
    # Convert image data and labels lists to numpy arrays
    image_combination_data = np.array(image_combination_data)
    image_combination_labels = np.array(image_combination_labels)
    
    return image_combination_data, image_combination_labels

In [3]:
# Randomly select a defined number of image paires from a given dataset
def get_right_min_image_combinations(number_image_pairs_selected, image_data, image_labels):
    
    num_images_total = len(image_labels)
    
    image_combination_data = []
    image_combination_labels = []
    
    for i in range(number_image_pairs_selected):
        
        # Draw two image indices from a uniform random distribution
        random_index_A = random.randint(0, num_images_total-1)
        random_index_B = random.randint(0, num_images_total-1)
        
        # Randomly choose two images from the dataset
        image_A = image_data[random_index_A]
        image_B = image_data[random_index_B]
        
        # Merge images TODO: this could be a function with several options
        combined_images = np.hstack((image_A, image_B))
        
        # Find the minimum between the two labels
        label_A = image_labels[random_index_A]
        label_B = image_labels[random_index_B]
        minimum_label = min(label_A, label_B)
        
        if label_A < label_B:
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_B, image_A))
        elif label_A > label_B: 
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_A, image_B))
        # Here we want to exclude the images where they are the same on both sides
            
   
        # Append newly generated image combination and minimum label to list
        image_combination_data.append(combined_images)
        image_combination_labels.append(minimum_label)
        
    # Convert image data and labels lists to numpy arrays
    image_combination_data = np.array(image_combination_data)
    image_combination_labels = np.array(image_combination_labels)
    
    return image_combination_data, image_combination_labels

In [4]:
# Load data and split into training and test
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [5]:
# Split training set further into training and validation sets
sample_size = 40000
X_train, X_val, y_train, y_val = train_images[:sample_size], train_images[sample_size:sample_size*2],  train_labels[:sample_size], train_labels[sample_size:sample_size*2]

In [6]:
combo_train_X, combo_train_y = get_left_min_image_combinations(40000, X_train, y_train)
combo_val_X, combo_val_y = get_right_min_image_combinations(20000, X_val, y_val)

In [7]:
# Reshape the data so they can be fed into the network, one-hot-encode the labels

train_images = combo_train_X.reshape((combo_train_X.shape[0], 28, 56, 1))
train_images = train_images/ 255

val_images = combo_val_X.reshape((combo_val_X.shape[0], 28, 56, 1))
val_images = val_images/255

train_labels = to_categorical(combo_train_y)
val_labels = to_categorical(combo_val_y)

In [8]:
# Model architecture
def build_CNN():

    # Convolutional NN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,56,1)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    # Adding a NN Classifier
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))

    model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy', 'mae'])

    return model

In [9]:
CNN_model = build_CNN()
CNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [10]:
# Fit model
CNN_model.fit(train_images, 
          train_labels, 
          epochs = 30, 
          batch_size = 64)

Epoch 1/30
625/625 [==============================] - 48s 75ms/step - loss: 0.1864 - accuracy: 0.9427 - mae: 0.0178
Epoch 2/30
625/625 [==============================] - 48s 77ms/step - loss: 0.0367 - accuracy: 0.9883 - mae: 0.0036
Epoch 3/30
625/625 [==============================] - 54s 86ms/step - loss: 0.0206 - accuracy: 0.9937 - mae: 0.0019
Epoch 4/30
625/625 [==============================] - 48s 76ms/step - loss: 0.0123 - accuracy: 0.9960 - mae: 0.0012
Epoch 5/30
625/625 [==============================] - 48s 77ms/step - loss: 0.0091 - accuracy: 0.9972 - mae: 7.9244e-04
Epoch 6/30
625/625 [==============================] - 51s 81ms/step - loss: 0.0063 - accuracy: 0.9979 - mae: 6.1226e-04
Epoch 7/30
625/625 [==============================] - 51s 82ms/step - loss: 0.0048 - accuracy: 0.9985 - mae: 4.3623e-04
Epoch 8/30
625/625 [==============================] - 50s 80ms/step - loss: 0.0039 - accuracy: 0.9988 - mae: 3.4045e-04
Epoch 9/30
625/625 [==============================] - 48

In [11]:
# Print loss, mae and accuracy
eval_model = CNN_model.evaluate(val_images, val_labels)
eval_model

625/625 [==============================] - 4s 6ms/step - loss: 82.2440 - accuracy: 0.1086 - mae: 0.1783


[82.24396514892578, 0.10864999890327454, 0.17834483087062836]

In [12]:
# Model architecture
def build_DNN():

    # Convolutional NN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,56,1)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    # Adding a NN Classifier
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))

    # Compile model
    model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy', 'mae'])

    return model

In [15]:
DNN_model = build_DNN()
DNN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1920)             

In [16]:
# Fit model
DNN_model.fit(train_images, 
          train_labels, 
          epochs = 30, 
          batch_size = 64)

Epoch 1/30
625/625 [==============================] - 68s 105ms/step - loss: 0.2147 - accuracy: 0.9323 - mae: 0.0206
Epoch 2/30
625/625 [==============================] - 73s 116ms/step - loss: 0.0570 - accuracy: 0.9822 - mae: 0.0054
Epoch 3/30
625/625 [==============================] - 69s 111ms/step - loss: 0.0387 - accuracy: 0.9879 - mae: 0.0036
Epoch 4/30
625/625 [==============================] - 66s 105ms/step - loss: 0.0300 - accuracy: 0.9908 - mae: 0.0027
Epoch 5/30
625/625 [==============================] - 61s 98ms/step - loss: 0.0234 - accuracy: 0.9923 - mae: 0.0021
Epoch 6/30
625/625 [==============================] - 63s 100ms/step - loss: 0.0201 - accuracy: 0.9938 - mae: 0.0018
Epoch 7/30
625/625 [==============================] - 66s 105ms/step - loss: 0.0171 - accuracy: 0.9944 - mae: 0.0015
Epoch 8/30
625/625 [==============================] - 61s 98ms/step - loss: 0.0137 - accuracy: 0.9955 - mae: 0.0012
Epoch 9/30
625/625 [==============================] - 60s 96ms/ste

In [17]:
# Print loss, mae and accuracy
DNN_eval_model = DNN_model.evaluate(val_images, val_labels)
DNN_eval_model

625/625 [==============================] - 5s 7ms/step - loss: 45.3017 - accuracy: 0.1077 - mae: 0.1785


[45.301734924316406, 0.10769999772310257, 0.1784740388393402]